In [6]:
import pyomo.kernel as pmo
import pandas as pd

from plant_opt.models.plant_model_stages_recourse_stochastic_pyo import Plant


# Problem Overview

Recommendation:
- Review `compare_recourse_nb.ipynb` to understand the background, as the model in this case is a more complex version of the model in that notebook.
- Review `plant_opt/models/plant_model_stages_pyo.py` to understand the deterministic version of the model, as the stochastic version builds on this model.

The model is a (deliberately non-realistic) refinery model that takes in crude oil (light or heavy) and produces a configurable number of intermediate and output products.
The refinery has a distillation unit that can take in a certain amount of crude oil to produce intermediate products, and refining units that can take in intermediate products to produce the final products.
The ratios of inputs to intermediates, and intermediates to outputs are parameters passed to the constructor of the model: any desired "production matrix" can be specified.

The model applies over "stages" in time (e.g. months), and the prices and demands for the final products are different in each stage.
There is a constraint applied to the model that the amount of each output product produced in each stage cannot change by more than a certain amount from the previous stage.
- This somewhat artificial constraint creates a time dependence, which makes the problem more interesting in the stochastic case, where the future is uncertain.

The model is designed to optimize the production of the final products in each stage given the constraints of the distillation and refining units as well as:
- the ratios of crude oil to intermediate products
- the ratios of intermediate products to final products.
- the prices of the inputs (light/heavy crude oil) and outputs.
- The "demands" for the final products: above a certain amount, the product no longer has value.

## Modelling Notes
The stochastic model uses a "scenario tree" of node objects, that each contain values for the variables that can change over time; as well as links to child nodes that represent the possible future scenarios from that point.
This model uses the pyomo kernel interface instead of the older "environ" interface, in order to create a model structure more closely associated with the scenario tree.
Non-anticipativity constraints are implicit: decision variables are only created at notes where they can actually take different values (for non-recourse decisions, they exist at the root node and all non-terminal nodes beneath it, for recourse decisions, they exist at all the non-root nodes - reflecting how the decision is made after the information at that stage is known).
All scenarios are assumed to be equally likely.

## Comparison to be made

To investigate the value of the stochastic model, we will perform a comparison by:
 - Solve the model given the full scenario tree and see the objective value
 - replace the final stage nodes of the scenario tree with nodes that have the variable values set to the average value of the variable in that stage.
 - solve the model with these average values.
 - take the solved value of the import variables at the root node from the "average" model and apply it as a fixed value in the root node of the stochastic model.
 - solve the model again with these initial values applied, and see the objective value.
The above can be repeated by going "up a level" in the scenario tree, creating an even more reduced model, and seeing how the objective value changes.



# Data Generation
The `random_walk_tree_builder` function in `plant_opt/scenario_tree/tree.py` creates scenario trees by applying a random (drawn from a gaussian distribution, and limited by a cap value) change to the values of the variables at each stage.
For this case, we will generate a tree with 4 stages, branching twice at each node.
By setting the "walk_max" value to zero for the crude prices and product prices, we keep them the same throughout the tree; changing only the demands.



In [3]:
from plant_opt.scenario_tree.tree import random_walk_tree_builder

stages = 4

variables = [
            "crude_light_price",
            "crude_heavy_price",
            "prod_price_0",
            "prod_price_1",
            "prod_price_2",
            "demand_0",
            "demand_1",
            "demand_2",
        ]

root, all_nodes = random_walk_tree_builder(
        variables,
        [30, 20, 50, 40, 30, 400, 300, 200],
        [1, 1, 1, 1, 1, 30, 30, 30],
        [0, 0, 0, 0, 0, 60, 60, 60],
        stages=stages,
        branch_factor=2,
        seed=42,
        truncate_places=0,
    )

Adding child (root_0) to parent=root
Now has children: ['root_0']
Adding child (root_1) to parent=root
Now has children: ['root_0', 'root_1']
Adding child (root_0_0) to parent=root_0
Now has children: ['root_0_0']
Adding child (root_0_1) to parent=root_0
Now has children: ['root_0_0', 'root_0_1']
Adding child (root_1_0) to parent=root_1
Now has children: ['root_1_0']
Adding child (root_1_1) to parent=root_1
Now has children: ['root_1_0', 'root_1_1']
Adding child (root_0_0_0) to parent=root_0_0
Now has children: ['root_0_0_0']
Adding child (root_0_0_1) to parent=root_0_0
Now has children: ['root_0_0_0', 'root_0_0_1']
Adding child (root_0_1_0) to parent=root_0_1
Now has children: ['root_0_1_0']
Adding child (root_0_1_1) to parent=root_0_1
Now has children: ['root_0_1_0', 'root_0_1_1']
Adding child (root_1_0_0) to parent=root_1_0
Now has children: ['root_1_0_0']
Adding child (root_1_0_1) to parent=root_1_0
Now has children: ['root_1_0_0', 'root_1_0_1']
Adding child (root_1_1_0) to parent=

The following cell creates the averaged/truncated scenario trees:

In [4]:
from collections import defaultdict
from copy import deepcopy

# Last stage averaged tree
one_averaged_tree = deepcopy(all_nodes)
# last two stages averaged tree
two_averaged_tree = deepcopy(all_nodes)

nodes_by_stage = defaultdict(list)
for node in one_averaged_tree:
    nodes_by_stage[node.stage].append(node)

stages_list = list(range(stages))

last_stage = stages_list[-1]

avg_var_values = {}
for var in variables:
    avg_var_values[var] = sum([node.values[var] for node in nodes_by_stage[last_stage]]) / len(nodes_by_stage[last_stage])

for node in nodes_by_stage[last_stage]:
    for var in variables:
        node.values[var] = avg_var_values[var]


nodes_by_stage = defaultdict(list)
for node in two_averaged_tree:
    nodes_by_stage[node.stage].append(node)

# Same as above, we also need to average the last stage for "two_averaged_tree"
for node in nodes_by_stage[last_stage]:
    for var in variables:
        node.values[var] = avg_var_values[var]

second_last_stage = stages_list[-2]
avg_var_values = {}
for var in variables:
    avg_var_values[var] = sum([node.values[var] for node in nodes_by_stage[second_last_stage]]) / len(nodes_by_stage[second_last_stage])

for node in nodes_by_stage[second_last_stage]:
    for var in variables:
        node.values[var] = avg_var_values[var]

print("Initial tree:")
for node in all_nodes:
    print(node.detail_str())

print("One stage averaged tree:")
for node in one_averaged_tree:
    print(node.detail_str())

print("Two stage averaged tree:")
for node in two_averaged_tree:
    print(node.detail_str())


Initial tree:
Node(root, None, 0, {'crude_light_price': 30, 'crude_heavy_price': 20, 'prod_price_0': 50, 'prod_price_1': 40, 'prod_price_2': 30, 'demand_0': 400, 'demand_1': 300, 'demand_2': 200})
Node(root_0, root, 1, {'crude_light_price': np.float64(30.0), 'crude_heavy_price': np.float64(20.0), 'prod_price_0': np.float64(50.0), 'prod_price_1': np.float64(40.0), 'prod_price_2': np.float64(30.0), 'demand_0': np.float64(361.0), 'demand_1': np.float64(304.0), 'demand_2': np.float64(191.0)})
Node(root_1, root, 1, {'crude_light_price': np.float64(30.0), 'crude_heavy_price': np.float64(20.0), 'prod_price_0': np.float64(50.0), 'prod_price_1': np.float64(40.0), 'prod_price_2': np.float64(30.0), 'demand_0': np.float64(434.0), 'demand_1': np.float64(314.0), 'demand_2': np.float64(174.0)})
Node(root_0_0, root_0, 2, {'crude_light_price': np.float64(30.0), 'crude_heavy_price': np.float64(20.0), 'prod_price_0': np.float64(50.0), 'prod_price_1': np.float64(40.0), 'prod_price_2': np.float64(30.0), 'd

## Model Solving

In [7]:
opt = pmo.SolverFactory("glpk")

In [8]:
# Some constants for all models
crude_distil_cap = 1000
products = 3
crude_ratios = [[3, 1, 0], [1, 2, 1], [0, 1, 2]]
refine_caps = [1000, 1000, 1000]
product_ratios = [
            [2, 1, 0.0],  # Amount of light product from l/m/h intermediates
            [0.2, 1.0, 0.2],  # Amount of medium product from l/m/h intermediates
            [0.0, 0.8, 2.0],  # Amount of heavy product from l/m/h intermediates
        ]
allowed_output_change = 20

In [9]:
p_full = Plant(
        crude_distil_cap=crude_distil_cap,
        products=products,
        crude_ratios=crude_ratios,
        refine_caps=refine_caps,
        product_ratios=product_ratios,
        stages=stages,
        scenario_tree_root=root,
        scenario_tree_all_nodes=all_nodes,
        allowed_output_change=allowed_output_change,
    )

p_one_averaged = Plant(
        crude_distil_cap=crude_distil_cap,
        products=products,
        crude_ratios=crude_ratios,
        refine_caps=refine_caps,
        product_ratios=product_ratios,
        stages=stages,
        scenario_tree_root=one_averaged_tree[0],
        scenario_tree_all_nodes=one_averaged_tree,
        allowed_output_change=allowed_output_change,
    )

p_two_averaged = Plant(
        crude_distil_cap=crude_distil_cap,
        products=products,
        crude_ratios=crude_ratios,
        refine_caps=refine_caps,
        product_ratios=product_ratios,
        stages=stages,
        scenario_tree_root=two_averaged_tree[0],
        scenario_tree_all_nodes=two_averaged_tree,
        allowed_output_change=allowed_output_change,
    )

In [13]:
# Basic solving of the models
result_df = pd.DataFrame(columns=["Model", "Objective Value", "Light Crude Import", "Heavy Crude Import"])

opt.solve(p_full.model)
result_df.loc[0] = ("Full Model", pmo.value(p_full.model.obj), pmo.value(p_full.model.light_crude_import[root]), pmo.value(p_full.model.heavy_crude_import[root]))

opt.solve(p_one_averaged.model)
one_avg_root = one_averaged_tree[0]
result_df.loc[1] = ("One Stage Averaged", pmo.value(p_one_averaged.model.obj), pmo.value(p_one_averaged.model.light_crude_import[one_avg_root]), pmo.value(p_one_averaged.model.heavy_crude_import[one_avg_root]))

opt.solve(p_two_averaged.model)
two_avg_root = two_averaged_tree[0]
result_df.loc[2] = ("Two Stage Averaged", pmo.value(p_two_averaged.model.obj), pmo.value(p_two_averaged.model.light_crude_import[two_avg_root]), pmo.value(p_two_averaged.model.heavy_crude_import[two_avg_root]))

Now apply the values from the averaged models to the full model (fixing the initial value) and solve again to determine the objective value:

In [16]:
p_full.model.light_crude_import[root].fixed = True
p_full.model.heavy_crude_import[root].fixed = True

p_full.model.light_crude_import[root].value = pmo.value(p_one_averaged.model.light_crude_import[one_avg_root])
p_full.model.heavy_crude_import[root].value = pmo.value(p_one_averaged.model.heavy_crude_import[one_avg_root])

opt.solve(p_full.model)
result_df.loc[3] = ("Full Model with One Stage Averaged", pmo.value(p_full.model.obj), pmo.value(p_full.model.light_crude_import[root]), pmo.value(p_full.model.heavy_crude_import[root]))

p_full.model.light_crude_import[root].value = pmo.value(p_two_averaged.model.light_crude_import[two_avg_root])
p_full.model.heavy_crude_import[root].value = pmo.value(p_two_averaged.model.heavy_crude_import[two_avg_root])

opt.solve(p_full.model)
result_df.loc[4] = ("Full Model with Two Stage Averaged", pmo.value(p_full.model.obj), pmo.value(p_full.model.light_crude_import[root]), pmo.value(p_full.model.heavy_crude_import[root]))

## Results

In [17]:
from IPython.display import display
display(result_df)

,Model,Objective Value,Light Crude Import,Heavy Crude Import
0,Full Model,102920.729167,17.333333,154.777778
1,One Stage Averaged,103054.801587,17.333333,154.777778
2,Two Stage Averaged,103285.970982,26.071429,138.785714
3,Full Model with One Stage Averaged,102920.729167,17.333333,154.777778
4,Full Model with Two Stage Averaged,102662.891865,26.071429,138.785714


Observations/Explanations:
- The objective value of the "averaged" models is higher, as the model is no longer attempting to account for uncertainty in the relevant stages
    - The model "thinks" that it can achieve higher value, which makes sense as there is no longer variation to account for
- The objective value of the full model with the "two stage averaged" values is lower, which makes sense, as the model is "locked in" to a suboptimal decision (w.r.t the variance in outcomes).
- Most interestingly, the objective value of the full model with the "one stage averaged" values is the exact same as the original full model:
    - This is expected once we observe that the light/heavy crude import values are the same in the "one stage averaged" model as in the original full model.
    - The implication here is that outcomes of the future stages are not having a large enough effect to matter at the first stage of the model.
        - In the real world, time/money could be saved by not considering the farther future stages in the model, as the model would be re-solved at each point in time anyways as outcomes were realized.
    - Future exploration could explore variations in the scenario tree to create situations where this is not the case - determine what circumstances are required for the future stages to matter at the first stage.